<img src="https://user-images.githubusercontent.com/39976464/41015148-6f6bd4a6-691f-11e8-9e39-3d62d40265e6.png">
# SER - 347 - Introdução à Programação para Sensoriamento Remoto 
## Operações Zonais Aplicadas a Dados de Luzes Noturnas
- Gabriel da Rocha Bragion
- Igor José Malfetoni Ferreira


## 1. Introdução 




### 2 Módulos utilizados





**2.1 OSGeo - Pacote de ferramentas do Python para manipulação de dados espaciais:**
    
- OGR - Biblioteca utilizada para manipulação de vetores;

- GDAL - Biblioteca utilizada para manipulação de raster;

- GDALconst - Constantes utilizadas para definir os tipos de dados nas categorias geoespaciais;
    
- GDAL Exceptions - Conjunto de exceções em casos de erro de processamento;

- OSR - Módulo para leitura de metadados.

In [1]:
from osgeo import gdal, ogr, osr
import gdal
gdal.UseExceptions()
from gdalconst import *


**2.2 Fiona - Pacote facilitador de ferramentas para leitura e escrita de dados vetoriais **
- Collection - Módulo voltado para leitura de coleções dados geoespaciais; 


In [2]:
import fiona
from fiona import collection

** 2.3 Collections - Pacote de coleções especializadas, como dicionários e listas alternativas **
- OrderedDict - Dicinário que memoriza a ordem das classes inseridas

In [3]:
import collections
from collections import OrderedDict

**2.4 Shapely - Pacote do Python para manipulação de geometrias planas **
- Shape - Permite o acesso e operações entre geometrias

In [4]:
import shapely as shp
from shapely.geometry import shape
import os


### 3. Manipulação de dados matriciais georreferenciados

- **Ojetivos**: 
    - Acessar um dado no formato GeoTiff e recuperar os seus metadados;
    - Organizar os metadados;
    - Poligonizar o raster (*Raster to Vector*)
    - Associar os metadados ao vetor criado (*shapefile*)
                        

** 3.1 Leitura e recuperação dos metadados **
 - Atribuição do raster a variável *viirs* e recuperação do Sistema de Referência de Coordenadas:

In [6]:
ds_raster = ("D:/trabalhoprog2/rec2_22s.tif")
viirs = gdal.Open(ds_raster)
srs = osr.SpatialReference()
srs.ImportFromWkt(viirs.GetProjection())



0

- Definindo Banda, nome e driver para criação do vetor de saída:

In [7]:
viirsband = viirs.GetRasterBand(1)

viirs_name = "D:/trabalhoprog2/viirspoly"

drv=ogr.GetDriverByName("ESRI Shapefile")

- Criando um shapefile de saída:

In [8]:

dst_ds = drv.CreateDataSource(viirs_name + ".shp")

dst_layer = dst_ds.CreateLayer(viirs_name, srs = srs)

newField = ogr.FieldDefn('DN', ogr.OFTReal)

dst_layer.CreateField(newField)

0


<img src="https://user-images.githubusercontent.com/39976464/41012564-79b750f4-6918-11e8-8afb-b1fd41ed7bef.png">
            - Figura 1: Shapefiles.
  

** 3.2 Poligonização  **

- A poligonização é o processo de transformar um raster em um vetor;
- Os pixels tangentes de mesmo valor são associados a uma única feição;
- A função retorna um shapefile contendo um conjunto de feições com vlores de atributos correspondentes aos do dado de entrada.

In [9]:
gdal.Polygonize(viirsband, None, dst_layer, 0, [], callback=None)
dst_ds=None

<img src="https://user-images.githubusercontent.com/39976464/41012585-93f01398-6918-11e8-8f4b-fd7fb6b56e13.png">
            - Figura 2: Função Poligonize: resultados.



### 4. Intersecção dos arquivos vetoriais

- **Ojetivos**: 
    - Acessar o shapefile correspondente aos setores censitários do município de Belém - PA;
    - Acessar os dados de luzes noturnas poligonizados na etapa anterior;
    - Calcular o número de pixels com valores de números digitais maior que cinco (5) para cada setor censitário. 
    - Criar um arquivo de saída no formato shapefile com o atributo "Números digitais maiores que 5 (ND)"
    
                        

**4.1 Leitura das geometrias dos setores censitários e associação dos metadados**

In [10]:
with fiona.open("D:/trabalhoprog2/set_para_4326_22s.shp", "r") as setores:
    source_driver = setores.driver
    source_crs = setores.crs
    source_schema = setores.schema

out_schema={'properties': OrderedDict([('ID', 'int:11'),
                                       ('CD_GEOCODI', 'str:20'),
                                       ('TIPO', 'str:10'), 
                                       ('CD_GEOCODB', 'str:20'),
                                       ('NM_BAIRRO', 'str:60'), 
                                       ('CD_GEOCODS', 'str:20'),
                                       ('NM_SUBDIST', 'str:60'), 
                                       ('CD_GEOCODD', 'str:20'),
                                       ('NM_DISTRIT', 'str:60'), 
                                       ('CD_GEOCODM', 'str:20'),
                                       ('NM_MUNICIP', 'str:60'), 
                                       ('NM_MICRO', 'str:100'),
                                       ('NM_MESO', 'str:100'),
                                       ('ND5', 'float:15.10')]),
            'geometry': 'Polygon'}




**4.2 Intersecção**
- Para setor censitário, será contabilizados o número de pixels válidos (ND>5)

    <img src="https://user-images.githubusercontent.com/39976464/41013647-8073cb7a-691d-11e8-8fa9-be1db856651e.png">
                                            - Figura 2: Função Poligonize: resultados.
            
    1. Criar um .shp de saída (intersec.shp)
    2. Abrir os dados vetoriais de entrada (setores; viirs)
    3. Verificar a intersecção entre cada setor (setores) e ntl (viirs)
    4. Calcular a área interctada
    5. Dividir o valor total intersectado pela área do pixel (número de pixels válidos)
    6. Associar o número de pixels válidos ao atributo ND, do novo .shp criado.

In [11]:
with fiona.open("D:/trabalhoprog2/interseccao.shp","w",
                driver=source_driver,
                crs=source_crs,
                schema=out_schema) as intersec:
    with fiona.open("D:/trabalhoprog2/set_para_4326_22s.shp", "r") as setores:
        with fiona.open("D:/trabalhoprog2/viirspoly.shp") as viirs:
            for setor in setores:
                geom_setor = shape(setor['geometry'])


                area_total = 0

                for ntl in viirs:
                    geom_viirs = shape(ntl['geometry'])
                    value_viirs = ntl['properties']['DN']
                    if geom_setor.intersects(geom_viirs) and value_viirs >0:
                        forma = geom_setor.intersection(geom_viirs)
                        area = forma.area
                        area_total = (area_total + area)
                setor['properties']['ND5'] = (area_total/215139.4153)
                intersec.write(setor)
                


## Considerações Finais 

- Poligonização de dados matriciais em vetoriais
- cálculo das áreas de intersecção das geometrias do arquivo vetorial criado; 

- Quantificação do número de pixels válidos associado a cada geometria interseccionada.

**No entanto, a aplicação do algoritmo em arquivos contendo um grande número de feições torna o processo lento. Dessa forma, projetos futuros devem investir esforços para a implementação de índices como Rtree a fim de otimizar o processo de verificação das geometrias, tornando o mais eficiente.**
